In [1]:
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd

In [2]:
### EXTRACTING PREVIOUSLY SAVED DATA FROM HDF5 FILE
### Declaring constants:
path_market_risk_source_hdf = 'Data_Files/Source_Files/market_risk_source.h5'
market_membership_key = 'market_membership_key'
realized_ret_USD_key = 'realized_ret_USD_key'
realized_ret_LOC_key = 'realized_ret_LOC_key'
index_ret_USD_key = 'index_ret_USD_key'
index_ret_LOC_key = 'index_ret_LOC_key'
ivol3m_key = 'ivol3m_key'
vrp3m_key = 'vrp3m_key'
path_gri_index_hdf = 'Data_Files/Source_Files/gri_released_index.h5'
gri_released_key = 'gri_released_key'
gri_start_key = 'gri_start_key'
gri_level_perc_key = 'gri_level_perc_key' ### ADDED FOR BETA FACTORS CALCULATION
gri_momentum_perc_key = 'gri_momentum_perc_key' ### ADDED FOR BETA FACTORS CALCULATION
### Exporting data from hdf5 fixed formatted files:
ser_market_membership = pd.read_hdf(path_market_risk_source_hdf, market_membership_key)
ser_realized_ret_USD = pd.read_hdf(path_market_risk_source_hdf, realized_ret_USD_key)
ser_realized_ret_LOC = pd.read_hdf(path_market_risk_source_hdf, realized_ret_LOC_key)
ser_index_ret_USD = pd.read_hdf(path_market_risk_source_hdf, index_ret_USD_key)
ser_index_ret_LOC = pd.read_hdf(path_market_risk_source_hdf, index_ret_LOC_key)
ser_ivol3m = pd.read_hdf(path_market_risk_source_hdf, ivol3m_key)
ser_vrp3m = pd.read_hdf(path_market_risk_source_hdf, vrp3m_key)
ser_gri_released = pd.read_hdf(path_market_risk_source_hdf, gri_released_key)
ser_gri_start_col = pd.read_hdf(path_market_risk_source_hdf, gri_start_key)
ser_gri_level_perc = pd.read_hdf(path_market_risk_source_hdf, gri_level_perc_key) ### ADDED FOR BETA FACTORS CALCULATION
ser_gri_momentum_perc = pd.read_hdf(path_market_risk_source_hdf, gri_momentum_perc_key) ### ADDED FOR BETA FACTORS CALCULATION

In [3]:
### EXTRACTING UNIVERSE DATA FOR A PARTICULAR DATE
def get_date_membership(iter_date):
    import numpy as np
    import pandas as pd    
    ### Preparing data for universe filtering:
    if (pd.to_datetime(iter_date) == pd.to_datetime(iter_date - pd.offsets.BusinessMonthEnd(0))):
        iter_month_end = iter_date
    else: 
        iter_month_end = pd.to_datetime(iter_date - pd.offsets.BusinessMonthEnd(1))    
#    ser_iter_membership = pd.read_hdf(path_market_risk_source_hdf, market_membership_key, where = 'Date = iter_month_end')
    ser_iter_membership = ser_market_membership.loc[:, iter_month_end : iter_month_end]
    ser_iter_membership.rename(index = {iter_month_end : iter_date}, inplace = True)
    
    return ser_iter_membership

In [4]:
### DEFINING EXPONENTIAL WEIGHTS GENERATOR
def get_exp_weights(window_years = 5, halflife_months = 3):
    ### Importing standard modules:    
    import numpy as np
    import pandas as pd
    import math     
    ### Defining constants:
    num_year_work_days = 260
    num_year_months = 12
    ### Array of regressioon window day numbers descending:
    arr_weight_days = np.arange(num_year_work_days * window_years, 0, -1) - 1
    ### Creating weights series:
    num_period_factor = math.exp(math.log(0.5) / round((num_year_work_days / num_year_months * halflife_months)))
    arr_weights = np.exp(math.log(num_period_factor) * arr_weight_days)
    ser_weights = pd.Series(arr_weights)        
    ser_weights.name = 'Weight'
    
    return ser_weights

In [5]:
### DEFINING WEIGHTS TO SERIES BINDER
def bind_exp_weights(ser_returns, weighting_kind = 'equal', window_years = 5, halflife_months = 3, ser_condition = pd.Series(np.NaN)):
    ### Importing standard modules:    
    import numpy as np
    import pandas as pd
    ### Creating weights series:
    if (weighting_kind == 'equal'):
        ser_weights = pd.Series(1, index = ser_returns.index)
    if (weighting_kind == 'expo'):       
        ser_weights = get_exp_weights(window_years, halflife_months)[- ser_returns.size : ]
        ser_weights.index = ser_returns.index
    if (weighting_kind == 'expo_cond_old'): ### BEFORE UDATING
        ser_condition = abs(ser_condition - ser_condition.iloc[-1])
        ser_condition = ser_condition.sort_values(ascending = False)
        ser_weights = get_exp_weights(window_years, halflife_months)[- ser_returns.size : ]        
        ser_weights = pd.Series(ser_weights.values, ser_condition.index)
        ser_weights.sort_index(inplace = True)
        ser_weights.name = 'Weight'
    if (weighting_kind == 'expo_cond'): ### AFTER UDATING
            index_sort = ser_returns.dropna().index
            ser_weights = get_exp_weights(window_years, halflife_months)[- ser_returns.size : ]
            ser_weights.index = ser_returns.index
            ser_cond_weights = pd.Series(np.NaN, ser_returns.index)
            df_to_sort = pd.concat([ser_returns, ser_condition], axis = 1)
            df_to_sort['Number'] = np.arange(ser_returns.size)
            df_to_sort.set_index('Number', inplace = True, drop = False)
            df_to_sort.dropna(inplace = True)
            if (df_to_sort['GRI'].count() > 0):
                df_to_sort['GRI'] = abs(df_to_sort['GRI'] - df_to_sort['GRI'].iloc[-1])
                df_to_sort.sort_values('GRI', inplace = True, ascending = False)
                df_to_sort['Date'] = index_sort.values
                df_to_sort.sort_index(inplace = True)           
                df_to_sort.set_index('Date', inplace = True)
                for (iter_weight_date, iter_weight_row) in df_to_sort.iterrows():
                    ser_cond_weights.iloc[int(iter_weight_row['Number'])] = ser_weights.loc[iter_weight_date]
        
    return ser_weights

In [6]:
### DEFINING EXPONENTIAL VOLATILITY CALCULATOR
def get_expvol_value(ser_returns, ser_weights):
    ### Importing standard modules:    
    import numpy as np
    import pandas as pd
    ### Exponential volatility calculating:
    expvol_result = np.NaN
    ser_returns = ser_returns.dropna()
    index_rolling = ser_returns.index.intersection(ser_weights.index)           
    ### Exponential volatility calculating:
    expvol_y = ser_returns[index_rolling]
    expvol_w = ser_weights[index_rolling]             
    expvol_w = expvol_w / expvol_w.sum()
    expvol_result = np.sqrt(expvol_w.dot(expvol_y * expvol_y))
        
    return expvol_result

In [7]:
### DEFINING EXPONENTIAL VOLATILITY SERIES BUILDER
def get_expvol_series(ser_market_membership, ser_returns, weighting_kind = 'equal', window_years = 5, halflife_months = 3, ser_condition = pd.Series(np.NaN)):
    ### Importing standard modules:    
    import numpy as np
    import pandas as pd
    ### Defining constants:
    num_year_work_days = 260
    ### Flattening MSCI changes by logarythm
    ser_returns = np.log(1 + ser_returns)
    ser_condition.fillna(method = 'ffill', inplace = True)
    ### Main loop performing:
    ser_expvol = pd.Series(np.NaN, index = ser_market_membership.index)
    for iter_country in ser_market_membership.index.get_level_values(0).unique():        
        ### Extracting returns data vector for each country/date point:
        if (iter_country in ser_returns.index.get_level_values(0).unique()):
            for iter_date in ser_market_membership[iter_country].index.get_level_values(0).unique():
                ser_iter_returns = ser_returns[iter_country].loc[iter_date - pd.offsets.BusinessDay(num_year_work_days * window_years - 1) : iter_date]
                ser_iter_returns.name = 'Returns'
                ser_iter_returns = ser_iter_returns - ser_iter_returns.mean()
                if (ser_iter_returns.size > 0):
                    if (ser_condition.count() > 0):
                        ser_iter_condition = ser_condition[ser_iter_returns.index]
                    else:
                        ser_iter_condition = pd.Series(np.NaN)                     
                    ser_iter_weights = bind_exp_weights(ser_iter_returns, weighting_kind, window_years, halflife_months, ser_iter_condition) ## CHANGES: ADDED       
                ser_iter_returns.dropna(inplace = True)
                if (ser_iter_returns.count() > num_year_work_days // 2):
                    expvol_result = get_expvol_value(ser_iter_returns, ser_iter_weights) * np.sqrt(num_year_work_days)
                    ser_expvol.loc[iter_country, iter_date] = expvol_result
    return ser_expvol

In [8]:
### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING 
### GET VOLATILITY SURPRISE FACTOR
def get_conditional_vol_factor(iter_date):
    ### Importing standard modules and date-special modules (common for all factors):
    import numpy as np
    import pandas as pd   
    from datetime import date
    ### Global parameters declaring:
    date_start = date(1993, 12, 31)     
    ### Constants declaring (common for all factors):  
    num_year_work_days = 260
    window_years = 5        
    ### Data preparing (common for all factors):
    index_iter_date = pd.date_range(end = iter_date, periods = num_year_work_days * window_years, freq = 'B')
    ser_iter_membership = get_date_membership(iter_date)  
    ser_iter_returns = ser_realized_ret_LOC.loc[:, index_iter_date]   
    ### Removing effectively missing return observations    
    ser_iter_returns.replace(0, np.nan, inplace = True)    
    ser_gri_added = pd.concat([ser_gri_start_col[date_start].loc[ : date_start - pd.offsets.BusinessDay()], ser_gri_released])
    ser_iter_condition = ser_gri_added[index_iter_date] 
    ### Factor calculation:
    ser_expvol1m_cond = get_expvol_series(ser_iter_membership, ser_iter_returns, 'expo_cond', window_years = window_years, halflife_months = 1, 
                                          ser_condition = ser_iter_condition)
    ser_iter_factor = ser_expvol1m_cond
    
    return ser_iter_factor

In [ ]:
### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING 

### LOOPER FOR CONDITIONAL VOLATILITY FACTOR
#date_range_test = pd.date_range(start = '2010-10-25', periods = 6)
date_range_test = ser_market_membership.sort_index(level = 1).index.get_level_values(1).unique()
#date_range_test = ser_market_membership.sort_index(level = 1).index.get_level_values(1).unique()[155 : 157]
arr_conditional_vol_factor = []
iter_counter = 0
for iter_date in date_range_test:
    iter_counter = iter_counter + 1
    arr_conditional_vol_factor.append(get_conditional_vol_factor(iter_date))
    if ((iter_counter // 10) == (iter_counter / 10)):
        print('Progress printout', iter_counter, '/', iter_date)
        
ser_conditional_vol_factor = pd.concat(arr_conditional_vol_factor)

In [10]:
### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING 

### CONDITIONAL VOLATILITY FACTOR TESTING:
print('ser_conditional_vol_factor - AR 29-Dec-2006:', ser_conditional_vol_factor.loc['AR' , '2006-12-29'])
print('ser_conditional_vol_factor - US 29-Dec-2006:', ser_conditional_vol_factor.loc['US' , '2006-12-29'])
ser_conditional_vol_factor_mean = pd.Series(np.NaN, index = ser_conditional_vol_factor.index.get_level_values(1).unique())
for iter_date in ser_conditional_vol_factor_mean.index:  
    ser_conditional_vol_factor_mean[iter_date] = ser_conditional_vol_factor.loc[:, iter_date].mean()
ser_conditional_vol_factor_mean.sort_index(inplace = True)
print('ser_conditional_vol_factor - cross-sectional mean min:', ser_conditional_vol_factor_mean.min())
print('ser_conditional_vol_factor - cross-sectional mean mean:', ser_conditional_vol_factor_mean.mean())
print('ser_conditional_vol_factor - cross-sectional mean max:', ser_conditional_vol_factor_mean.max())
print('ser_conditional_vol_factor - cross-sectional mean stdev:', ser_conditional_vol_factor_mean.std())
print('ser_conditional_vol_factor - cross-sectional mean count:', ser_conditional_vol_factor_mean.count())

ser_conditional_vol_factor - AR 29-Dec-2006: 0.2107838560995583
ser_conditional_vol_factor - US 29-Dec-2006: 0.0817462646666443
ser_conditional_vol_factor - cross-sectional mean min: 0.12735794074386794
ser_conditional_vol_factor - cross-sectional mean mean: 0.2140489750963062
ser_conditional_vol_factor - cross-sectional mean max: 0.6762556480844379
ser_conditional_vol_factor - cross-sectional mean stdev: 0.07974120939017841
ser_conditional_vol_factor - cross-sectional mean count: 234


In [50]:
### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING 
### Importing standard modules and date-special modules (common for all factors):
import numpy as np
import pandas as pd   
from datetime import date
### Constants declaring:
date_start = date(1993, 12, 31)
num_year_work_days = 260
window_years = 5  
### Testing:
#date_range_test = ser_market_membership.sort_index(level = 1).index.get_level_values(1).unique()
iter_date = date(2002, 2, 28)
date_range_test = pd.date_range(end = iter_date, periods = 1, freq = 'B')
ser_result = pd.Series(np.NaN, index = date_range_test)
for iter_date in date_range_test:
    print(iter_date)
#get_conditional_vol_factor(iter_date).loc['US']
### ENCODING:
    ### Data preparing (common for all factors):
    index_iter_date = pd.date_range(end = iter_date, periods = num_year_work_days * window_years, freq = 'B')
    ser_iter_membership = get_date_membership(iter_date)  
    ser_iter_returns = ser_realized_ret_LOC.loc[:, index_iter_date]   
    ### Removing effectively missing return observations    
    ser_iter_returns.replace(0, np.nan, inplace = True)    
    ser_gri_added = pd.concat([ser_gri_start_col[date_start].loc[ : date_start - pd.offsets.BusinessDay()], ser_gri_released])
    ser_iter_condition = ser_gri_added[index_iter_date] 

    ### Factor calculation:
    #get_expvol_series(ser_iter_membership, ser_iter_returns, 'expo_cond', window_years = window_years, halflife_months = 1, ser_condition = ser_iter_condition).loc['US']
    ### ENCODING:
    ### Flattening MSCI changes by logarythm
    ser_returns = ser_iter_returns.copy()
    ser_condition = ser_iter_condition.copy()
    weighting_kind = 'expo_cond'
    halflife_months = 1
    ser_returns = np.log(1 + ser_returns)
    ser_condition.fillna(method = 'ffill', inplace = True)
    ### Main loop performing:
    iter_country = 'US'
    ser_iter_returns = ser_returns[iter_country].loc[iter_date - pd.offsets.BusinessDay(num_year_work_days * window_years - 1) : iter_date]
    ser_iter_returns = ser_iter_returns - ser_iter_returns.mean()
    if (ser_iter_returns.size > 0):
        ser_iter_condition = ser_condition[ser_iter_returns.index]

        if (ser_iter_condition.count() > 0):
            
            
            ser_iter_returns = ser_iter_returns.reset_index(drop = True)
            ser_iter_condition = ser_iter_condition.reset_index(drop = True)
            ser_iter_weights = get_exp_weights(window_years, halflife_months)[- ser_iter_returns.size : ]
            ser_iter_weights.index = ser_iter_returns.index

            ser_iter_returns.dropna(inplace = True)
            ser_iter_condition = ser_iter_condition[ser_iter_returns.index]
            ser_iter_weights = ser_iter_weights[ser_iter_returns.index]
            ser_iter_returns = ser_iter_returns.reset_index(drop = True)
            if (ser_iter_condition.count() > 0):
                ser_iter_returns.name = 'Returns'
                ser_iter_condition = ser_iter_condition.reset_index(drop = True)
                ser_iter_weights = ser_iter_weights.reset_index(drop = True)
                ser_iter_condition = abs(ser_iter_condition - ser_iter_condition.iloc[-1])
                df_sorting = pd.concat([ser_iter_condition, ser_iter_returns], axis = 1)
                df_sorting.sort_values('GRI', inplace = True, ascending = False)
                ser_iter_returns = df_sorting['Returns']
                ser_iter_returns = ser_iter_returns.reset_index(drop = True)

                expvol_y = ser_iter_returns
                expvol_w = ser_iter_weights                   
                expvol_w = expvol_w / expvol_w.sum()
                expvol_result = np.sqrt(expvol_w.dot(expvol_y * expvol_y)) 

                expvol_result = expvol_result * np.sqrt(num_year_work_days)
                ser_result[iter_date] = expvol_result
                
        else:
            ser_iter_condition = pd.Series(np.NaN)  

2002-02-28 00:00:00


In [11]:
ser_result

2002-02-28    0.175407
Freq: B, dtype: float64

In [19]:
### TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING TESTING 
### Importing standard modules and date-special modules (common for all factors):
import numpy as np
import pandas as pd   
from datetime import date
### Constants declaring:
date_start = date(1993, 12, 31)
num_year_work_days = 260
window_years = 5  
### Testing:
date_range_test = ser_market_membership.sort_index(level = 1).index.get_level_values(1).unique()
#iter_date = date(2002, 2, 28)
#date_range_test = pd.date_range(end = iter_date, periods = 1, freq = 'B')
ser_result = pd.Series(np.NaN, index = date_range_test)
for iter_date in date_range_test:
    if iter_date.month == 12:
        print(iter_date)
#get_conditional_vol_factor(iter_date).loc['US']
### ENCODING:
    ### Data preparing (common for all factors):
    index_iter_date = pd.date_range(end = iter_date, periods = num_year_work_days * window_years, freq = 'B')
    ser_iter_membership = get_date_membership(iter_date)  
    ser_iter_returns = ser_realized_ret_LOC.loc[:, index_iter_date]   
    ### Removing effectively missing return observations    
    ser_iter_returns.replace(0, np.nan, inplace = True)    
    ser_gri_added = pd.concat([ser_gri_start_col[date_start].loc[ : date_start - pd.offsets.BusinessDay()], ser_gri_released])
    ser_iter_condition = ser_gri_added[index_iter_date] 

    ### Factor calculation:
    #get_expvol_series(ser_iter_membership, ser_iter_returns, 'expo_cond', window_years = window_years, halflife_months = 1, ser_condition = ser_iter_condition).loc['US']
    ### ENCODING:
    ### Flattening MSCI changes by logarythm
    ser_returns = ser_iter_returns.copy()
    ser_condition = ser_iter_condition.copy()
    weighting_kind = 'expo_cond'
    halflife_months = 1
    ser_returns = np.log(1 + ser_returns)
    ser_condition.fillna(method = 'ffill', inplace = True)
    ### Main loop performing:
    iter_country = 'US'
    ser_iter_returns = ser_returns[iter_country].loc[iter_date - pd.offsets.BusinessDay(num_year_work_days * window_years - 1) : iter_date]
    ser_iter_returns.name = 'Returns'
    ser_iter_returns = ser_iter_returns - ser_iter_returns.mean()
    if (ser_iter_returns.size > 0):
        ser_iter_condition = ser_condition[ser_iter_returns.index]

        if (ser_iter_condition.count() > 0):

            index_iter_sort = ser_iter_returns.dropna().index
            ser_iter_weights = get_exp_weights(window_years, halflife_months)[- ser_iter_returns.size : ]
            ser_iter_weights.index = ser_iter_returns.index
            ser_cond_weights = pd.Series(np.NaN, ser_iter_returns.index)
            df_to_sort = pd.concat([ser_iter_returns, ser_iter_condition], axis = 1)
            df_to_sort['Number'] = np.arange(ser_iter_returns.size) #
##            df_to_sort = df_to_sort.reset_index() ##
            df_to_sort.set_index('Number', inplace = True, drop = False) #
            df_to_sort.dropna(inplace = True)
            if (df_to_sort['GRI'].count() > 0):
                df_to_sort['GRI'] = abs(df_to_sort['GRI'] - df_to_sort['GRI'].iloc[-1])
                df_to_sort.sort_values('GRI', inplace = True, ascending = False)
                df_to_sort['Date'] = index_iter_sort.values
                df_to_sort.sort_index(inplace = True)           
                df_to_sort.set_index('Date', inplace = True) #          
                df_to_sort['Number'] = df_to_sort['Number'].astype(np.int16)

                for (iter_weight_date, iter_weight_row) in df_to_sort.iterrows():
                    ser_cond_weights.iloc[int(iter_weight_row['Number'])] = ser_iter_weights.loc[iter_weight_date]

                ser_iter_returns = ser_iter_returns.dropna()
                index_rolling = ser_iter_returns.index.intersection(ser_cond_weights.index)           
                ### Exponential volatility calculating:
                expvol_y = ser_iter_returns[index_rolling]
                expvol_w = ser_cond_weights[index_rolling]             
                expvol_w = expvol_w / expvol_w.sum()
                expvol_result = np.sqrt(expvol_w.dot(expvol_y * expvol_y))

                expvol_result = expvol_result * np.sqrt(num_year_work_days)
                ser_result[iter_date] = expvol_result

###            df_to_sort.columns = df_to_sort.columns + '_sort'
            
###            df_to_rew = pd.concat([ser_iter_returns, ser_iter_weights], axis = 1)
##            df_to_rew = df_to_rew.reset_index() ##
###            df_to_rew['Number'] = np.arange(ser_iter_returns.size) #
##            df_to_rew.set_index('Date', inplace = True) ##
###            df_to_rew.loc[df_to_rew['Returns'].isna(), 'Weight'] = 0
###            df_to_rew.columns = df_to_rew.columns + '_rew'
            
                
        else:
            ser_iter_condition = pd.Series(np.NaN)  

1994-12-30 00:00:00
1995-12-29 00:00:00
1996-12-31 00:00:00
1997-12-31 00:00:00
1998-12-31 00:00:00
1999-12-31 00:00:00
2000-12-29 00:00:00
2001-12-31 00:00:00
2002-12-31 00:00:00
2003-12-31 00:00:00
2004-12-31 00:00:00
2005-12-30 00:00:00
2006-12-29 00:00:00
2007-12-31 00:00:00
2008-12-31 00:00:00
2009-12-31 00:00:00
2010-12-31 00:00:00
2011-12-30 00:00:00
2012-12-31 00:00:00
2013-12-31 00:00:00
2014-12-31 00:00:00
2015-12-31 00:00:00
2016-12-30 00:00:00
2017-12-29 00:00:00
2018-12-31 00:00:00


In [ ]:
ser_result

In [20]:
ser_result.to_excel('Data_Files/Test_Files/conditional_testing_plus.xlsx')

In [194]:
df_common = pd.concat([df_to_sort, df_to_rew], axis = 1)
df_common.tail(10)

,Returns,GRI,Number,Returns,Weight,Number
Date,,,,,,
2002-02-15,-0.012166,0.252743,1290.0,-0.012166,0.753098,1290
2002-02-18,NaN,NaN,NaN,NaN,0.000000,1291
2002-02-19,0.008513,0.199892,1295.0,-0.018764,0.802080,1292
2002-02-20,-0.015544,0.197225,1294.0,0.013783,0.827753,1293
2002-02-21,-0.018764,0.196251,1292.0,-0.015544,0.854248,1294
2002-02-22,0.013783,0.195641,1293.0,0.008513,0.881591,1295
2002-02-25,0.017975,0.183550,1296.0,0.017975,0.909809,1296
2002-02-26,-0.000155,0.123948,1297.0,-0.000155,0.938931,1297
2002-02-27,0.000774,0.068722,1298.0,0.000774,0.968984,1298
